# import the data

$$
f=x^2 + \sqrt {y} + z
$$

In [704]:
import pandas as pd

In [705]:
df = pd.read_csv("data/result_testdata.csv", delimiter=",")
df

,x,y,z,f
0,0.876300,0.087160,0.678294,1.741426
1,0.506543,0.427771,0.117943,1.028571
2,0.469181,0.208947,0.862431,1.539668
3,0.997632,0.038999,0.628393,1.821145
4,0.680144,0.432733,0.571025,1.691445
...,...,...,...,...
569,0.213704,0.183989,0.577722,1.052330
570,0.727660,0.021602,0.820556,1.497021
571,0.452470,0.848223,0.896740,2.022459
572,0.470553,0.443268,0.204643,1.091847


# build the network

https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/

In [706]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch import autograd
import math
from sklearn.metrics import mean_squared_error

In [707]:
X = df.iloc[:,:3]
y = df.iloc[:,3]
# y = df.iloc[:,:3].sum(axis=1) 

In [708]:
X = torch.tensor(X.values.astype(np.float32), requires_grad=True)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

In [709]:
X.shape

torch.Size([574, 3])

In [710]:
y.shape

torch.Size([574, 1])

In [711]:
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(3, 160)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(160, 80)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(80, 16)
        self.act_output =  nn.Linear(16, 1)
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x
 
model = PimaClassifier()
print(model)

PimaClassifier(
  (hidden1): Linear(in_features=3, out_features=160, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=160, out_features=80, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=80, out_features=16, bias=True)
  (act_output): Linear(in_features=16, out_features=1, bias=True)
)


In [713]:
loss_fn = nn.MSELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 500
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size].clone()
        # Xbatch.requires_grad=True
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        f_x_t = autograd.grad(y_pred,Xbatch,torch.ones([Xbatch.shape[0], 1]), retain_graph=True, create_graph=True)[0] #first derivative
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch==n_epochs-1 and i==0:
            print(Xbatch)
            print(ybatch)
            print(y_pred)
            print(f_x_t)
            print(f'Finished epoch {epoch}, latest loss {loss}')

tensor([[0.8763, 0.0872, 0.6783],
        [0.5065, 0.4278, 0.1179],
        [0.4692, 0.2089, 0.8624],
        [0.9976, 0.0390, 0.6284],
        [0.6801, 0.4327, 0.5710],
        [0.5288, 0.2589, 0.6544],
        [0.7432, 0.7144, 0.2645],
        [0.3437, 0.6037, 0.3529],
        [0.1806, 0.1608, 0.3050],
        [0.9655, 0.0042, 0.8323]], grad_fn=<CloneBackward0>)
tensor([[1.7414],
        [1.0286],
        [1.5397],
        [1.8211],
        [1.6914],
        [1.4429],
        [1.6620],
        [1.2481],
        [0.7386],
        [1.8292]])
tensor([[1.7439],
        [1.0362],
        [1.5405],
        [1.8132],
        [1.6917],
        [1.4408],
        [1.6610],
        [1.2499],
        [0.7446],
        [1.8582]], grad_fn=<AddmmBackward0>)
tensor([[1.8583, 2.2220, 1.0891],
        [0.9633, 0.7823, 0.8988],
        [0.9722, 1.1291, 0.9991],
        [1.8605, 2.1592, 0.9840],
        [1.2633, 0.7933, 0.9584],
        [1.0643, 0.8607, 0.9742],
        [1.5625, 0.5973, 1.0075],
       

In [714]:
# compute accuracy (no_grad is optional)
# with torch.no_grad():
#     y_pred = model(X)

X.requires_grad_()
X.retain_grad()
y_pred = model(X)

trainScore = math.sqrt(mean_squared_error(y_pred.detach().numpy(),y.detach().numpy()))
print('Train Score: %.2f RMSE' % (trainScore))

Train Score: 0.01 RMSE


# test the auto gradient

$$
f=x^2 + \sqrt {y} + z
$$

$$
f_x'=2x
$$

$$
f_y'=\frac{1}{2} * y ^ {-\frac{1}{2}}
$$

$$
f_z'=1
$$

In [715]:
X[0]

tensor([0.8763, 0.0872, 0.6783], grad_fn=<SelectBackward0>)

In [716]:
y[0]

tensor([1.7414])

In [725]:
model(X[0])

tensor([1.7590], grad_fn=<AddBackward0>)

In [724]:
autograd.grad(model(X),X,torch.ones([X.shape[0], 1]), retain_graph=True, create_graph=True)[0][0] #first derivative

tensor([1.8718, 2.0428, 1.0036], grad_fn=<SelectBackward0>)

good match!